# MusicExpander Prototype 2

In [ ]:
# get a huge collection of song info from spotify:
#including song_name, artist_name, year, href link, features

## Import libraries

In [4]:
#import libraries
from bs4 import BeautifulSoup
import pandas as pd
import requests
from random import randrange
import spotipy # install if needed
from spotipy.oauth2 import SpotifyClientCredentials

## Add YOUR Spotity credentials

In [5]:
#Initialize SpotiPy with user credentials
# Remove mine when uploading

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id="9f9d46aed10c480b9b68543a2d15e2c9",
    client_secret="0016f56c1cc641159dc9e19ba85bc682"))

## Functions:

In [119]:
#get a huge collection of song info from spotify:
#first create a collection of song names:
#get playlist, extranct song uris:
 #uri of a playlist with 100 songs from different artists:
ironhack_tunes="spotify:playlist:1kVI7R6OrKNN8Bf8GMJq9q"
shower_playlist="spotify:playlist:37i9dQZF1DWSqmBTGDYngZ"
eighties_to_oughts_playlist="spotify:playlist:6xnhcezDUVM4q6CHbX0nTS"
longest_playlist_ever="spotify:playlist:6yPiKpy7evrwvZodByKvM9"
long_playlist="spotify:playlist:0twlW4iDRhKXAJPopD8PP0"


In [195]:
#starting again:
def get_info_from_playlist(playlist_name):

    '''
    get name of song, artists, and uri of the song in a df
    '''
    
    results = sp.user_playlist_tracks("spotify", playlist_name)
    items = results['items']
    song_uris=[]
    songs = []
    artists = []
    song_id=[]
    
    #the the first 100 songs
    for song_item in items:
             # print(["track"][0].keys())
            songs.append(song_item["track"]["name"])
            artists.append(song_item["track"]["artists"][0]["name"])
            song_uris.append(song_item["track"]["uri"])
            song_id.append(song_item["track"]["uri"][14:])
    
    #while there is a next, keep collecting songs
    while results['next']:
        results = sp.next(results)
        items.extend(results['items'])
        for song_item in items:
            songs.append(song_item["track"]["name"])
            artists.append(song_item["track"]["artists"][0]["name"])
            song_uris.append(song_item["track"]["uri"])
            song_id.append(song_item["track"]["uri"][14:])
        
    #create a dictionary after the loop:
    songs_dict={
    "song titles":songs,
    "artists":artists,
    "songs uri":song_uris,
    "song id":song_id
    }

    #Each key becomes a dataframe column name:
    library_df=pd.DataFrame(songs_dict)
    
    return library_df



def get_song_features(song_uri):
    '''
    input a individual song uri
    output a df with a single row,
    and the features in columns
    '''
    features_dict=sp.audio_features(song_uri)
    features_df=pd.DataFrame(features_dict)
    
    return features_df

def get_uris(playlist_name):
    '''
    input a playlist
    output a list of song uris
    the output can be feeded to the function get_song_features
    '''
    #include the while
    playlist = sp.user_playlist_tracks("spotify", playlist_name)
    items = playlist["items"]
    song_uris = []
    
    for song in items:
        song_uris.append(song["track"]["uri"][14:])
    return song_uris


def get_uris_all(playlist_name):
    #with next:
    '''
    input a playlist
    output a list of song uris
    the output can be feeded to the function get_song_features
    '''
    #include the while
    playlist = sp.user_playlist_tracks("spotify", playlist_name)
    items = playlist["items"]
    song_uris = []
    
    for song_item in items:
        song_uris.append(song_item["track"]["uri"][14:])
    
    #while there is a next, keep collecting songs
    while playlist['next']:
        playlist = sp.next(playlist)
        items.extend(playlist['items'])
        for song_item in items:
            song_uris.append(song_item["track"]["uri"][14:])
        
    return song_uris


In [196]:
# Build the super playlist:

#List of playlist to include:
list_of_playlists=[ironhack_tunes, shower_playlist, eighties_to_oughts_playlist,longest_playlist_ever]

list_of_df=[]

#append the df with the info (song names, artist and song uri)
#to a list of df:
for l in list_of_playlists:
    list_of_df.append(get_info_from_playlist(l))

#concat the df in a big df:
df_total_pl=pd.concat(list_of_df)


#drop duplicates:
final_df=df_total_pl.drop_duplicates()
len(final_df)

#from the final_df I should extract the column song uri and pass it to get_song features.

10469

In [197]:
final_df.head()

,song titles,artists,songs uri,song id
0,D.A.N.C.E,Justice,spotify:track:33yAEqzKXexYM3WlOYtTfQ,33yAEqzKXexYM3WlOYtTfQ
1,"White & Nerdy (Parody of ""Ridin'"" by Chamillio...","""Weird Al"" Yankovic",spotify:track:60R2v9lheAu3lwZwAFxMZK,60R2v9lheAu3lwZwAFxMZK
2,Goliath,Smith & Thell,spotify:track:0z4kyKuFGmUV7WQ7gB1sAp,0z4kyKuFGmUV7WQ7gB1sAp
3,L'Empordà,Sopa De Cabra,spotify:track:1vzugDTXDgZCF0QQrzB5hX,1vzugDTXDgZCF0QQrzB5hX
4,House of the Rising Sun,The Animals,spotify:track:4mn2kNTqiGLwaUR8JdhJ1l,4mn2kNTqiGLwaUR8JdhJ1l


In [216]:
#DataFrame.drop(labels=None, axis=0, index=None, columns=None, level=None, inplace=False, errors='raise')
A=final_df[final_df['song id'].map(len) == 22]
#clean_df=final_df.drop()
A.head()

,song titles,artists,songs uri,song id
0,D.A.N.C.E,Justice,spotify:track:33yAEqzKXexYM3WlOYtTfQ,33yAEqzKXexYM3WlOYtTfQ
1,"White & Nerdy (Parody of ""Ridin'"" by Chamillio...","""Weird Al"" Yankovic",spotify:track:60R2v9lheAu3lwZwAFxMZK,60R2v9lheAu3lwZwAFxMZK
2,Goliath,Smith & Thell,spotify:track:0z4kyKuFGmUV7WQ7gB1sAp,0z4kyKuFGmUV7WQ7gB1sAp
3,L'Empordà,Sopa De Cabra,spotify:track:1vzugDTXDgZCF0QQrzB5hX,1vzugDTXDgZCF0QQrzB5hX
4,House of the Rising Sun,The Animals,spotify:track:4mn2kNTqiGLwaUR8JdhJ1l,4mn2kNTqiGLwaUR8JdhJ1l


In [223]:
#A=final_df[final_df['song id'].map(len) == 22]
A=final_df[~final_df['song id'].str.contains(r':')]

In [224]:
len(A)

10387

In [225]:
B=A[A['song id'].map(len) == 22]
len(B)

10387

In [ ]:
'''
Arcade+Fire:Funeral:Neighborhood+%231+%28Tunnels%29:288
The+Black+Keys:Turn+Blue:Bullet+In+the+Brain:256
The+Black+Keys:Turn+Blue:Fever:246
The+Black+Keys:Turn+Blue:Gotta+Get+Away:182
The+Black+Keys:Turn+Blue:In+Our+Prime:278
'''

In [226]:
#from the final_df I should extract the column song uri and pass it to get_song features.

song_uris_list=B["song id"].tolist()
list_of_df_names=[]
for uri in song_uris_list:
    #append each df (a single row each), to a list
    list_of_df_names.append(get_song_features(uri))
    


In [227]:
#concat all the dataframes, since the index of each df is 0, ignore index
df_total_features=pd.concat(list_of_df_names, ignore_index=True) 

In [228]:
len(df_total_features)

10387

In [229]:
df_total_features.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.613,0.962,6,-5.070,0,0.2780,0.034000,0.000000,0.8240,0.496,113.015,audio_features,33yAEqzKXexYM3WlOYtTfQ,spotify:track:33yAEqzKXexYM3WlOYtTfQ,https://api.spotify.com/v1/tracks/33yAEqzKXexY...,https://api.spotify.com/v1/audio-analysis/33yA...,242267,4
1,0.791,0.613,1,-11.628,0,0.0763,0.098600,0.000000,0.0765,0.896,143.017,audio_features,60R2v9lheAu3lwZwAFxMZK,spotify:track:60R2v9lheAu3lwZwAFxMZK,https://api.spotify.com/v1/tracks/60R2v9lheAu3...,https://api.spotify.com/v1/audio-analysis/60R2...,170640,4
2,0.528,0.833,9,-6.190,1,0.0514,0.000414,0.000001,0.0680,0.460,127.998,audio_features,0z4kyKuFGmUV7WQ7gB1sAp,spotify:track:0z4kyKuFGmUV7WQ7gB1sAp,https://api.spotify.com/v1/tracks/0z4kyKuFGmUV...,https://api.spotify.com/v1/audio-analysis/0z4k...,204759,4
3,0.681,0.837,4,-8.907,1,0.0410,0.036200,0.000002,0.0986,0.959,143.472,audio_features,1vzugDTXDgZCF0QQrzB5hX,spotify:track:1vzugDTXDgZCF0QQrzB5hX,https://api.spotify.com/v1/tracks/1vzugDTXDgZC...,https://api.spotify.com/v1/audio-analysis/1vzu...,159333,4
4,0.316,0.484,9,-9.110,0,0.0308,0.000334,0.004450,0.0912,0.299,117.363,audio_features,4mn2kNTqiGLwaUR8JdhJ1l,spotify:track:4mn2kNTqiGLwaUR8JdhJ1l,https://api.spotify.com/v1/tracks/4mn2kNTqiGLw...,https://api.spotify.com/v1/audio-analysis/4mn2...,269907,3


In [230]:
df_total_features.to_csv('total_features.csv')

In [231]:
B.to_csv('total_playlist_info.csv')

In [233]:
df_complete=B.merge(df_total_features, left_on='songs uri', right_on='uri')

In [234]:
df_complete.head()

,song titles,artists,songs uri,song id,danceability,energy,key,loudness,mode,speechiness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,D.A.N.C.E,Justice,spotify:track:33yAEqzKXexYM3WlOYtTfQ,33yAEqzKXexYM3WlOYtTfQ,0.613,0.962,6,-5.070,0,0.2780,...,0.8240,0.496,113.015,audio_features,33yAEqzKXexYM3WlOYtTfQ,spotify:track:33yAEqzKXexYM3WlOYtTfQ,https://api.spotify.com/v1/tracks/33yAEqzKXexY...,https://api.spotify.com/v1/audio-analysis/33yA...,242267,4
1,"White & Nerdy (Parody of ""Ridin'"" by Chamillio...","""Weird Al"" Yankovic",spotify:track:60R2v9lheAu3lwZwAFxMZK,60R2v9lheAu3lwZwAFxMZK,0.791,0.613,1,-11.628,0,0.0763,...,0.0765,0.896,143.017,audio_features,60R2v9lheAu3lwZwAFxMZK,spotify:track:60R2v9lheAu3lwZwAFxMZK,https://api.spotify.com/v1/tracks/60R2v9lheAu3...,https://api.spotify.com/v1/audio-analysis/60R2...,170640,4
2,Goliath,Smith & Thell,spotify:track:0z4kyKuFGmUV7WQ7gB1sAp,0z4kyKuFGmUV7WQ7gB1sAp,0.528,0.833,9,-6.190,1,0.0514,...,0.0680,0.460,127.998,audio_features,0z4kyKuFGmUV7WQ7gB1sAp,spotify:track:0z4kyKuFGmUV7WQ7gB1sAp,https://api.spotify.com/v1/tracks/0z4kyKuFGmUV...,https://api.spotify.com/v1/audio-analysis/0z4k...,204759,4
3,L'Empordà,Sopa De Cabra,spotify:track:1vzugDTXDgZCF0QQrzB5hX,1vzugDTXDgZCF0QQrzB5hX,0.681,0.837,4,-8.907,1,0.0410,...,0.0986,0.959,143.472,audio_features,1vzugDTXDgZCF0QQrzB5hX,spotify:track:1vzugDTXDgZCF0QQrzB5hX,https://api.spotify.com/v1/tracks/1vzugDTXDgZC...,https://api.spotify.com/v1/audio-analysis/1vzu...,159333,4
4,House of the Rising Sun,The Animals,spotify:track:4mn2kNTqiGLwaUR8JdhJ1l,4mn2kNTqiGLwaUR8JdhJ1l,0.316,0.484,9,-9.110,0,0.0308,...,0.0912,0.299,117.363,audio_features,4mn2kNTqiGLwaUR8JdhJ1l,spotify:track:4mn2kNTqiGLwaUR8JdhJ1l,https://api.spotify.com/v1/tracks/4mn2kNTqiGLw...,https://api.spotify.com/v1/audio-analysis/4mn2...,269907,3


In [235]:
df_complete.to_csv('greatest_df.csv')